# Regression.ipynb
Description   : Single-layer neural network for regression

## 회귀를 위한 신경망 설계
신경망을 직접 구현하여 회귀를 활용해보기로 한다.<br>
많은 예시로 사용되고 있는 전복 자료를 통한 단층 퍼셉트론을 구현해보자.

In [1]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import os

np.random.seed(1024)
# 경로 초기화
os.chdir(r"C:\Users\TitusChoi\Desktop\Library\CodeLion\AI")

In [2]:
# Hyperparameter
RND_MEAN = 0
RND_STD = 0.003
LEARNING_RATE = 0.001

In [3]:
# Main function
def regression_exec(epochs = 10, mb_size = 10, report = 1, tr = 0.7): # 학습 횟수, 미니 배치 사이즈, 학습 리포트, 훈련 비율(데이터 전부 훈련시키지 않고 그 중 훈련 비율만큼 훈련, 나머지는 시험 비율이 된다.)
    regression_load_dataset() # 데이터 셋 여는 함수
    init_model() # 가중치와 편향 초기화 함수
    train_and_test(epochs, mb_size, report, tr) # 학습 및 신경망 성능 테스트 함수

In [4]:
# Data load, one-hot encoding
def regression_load_dataset():
    # Loading datasets
    with open('./datasets/abalone.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None) # 첫 번째 index 건너뛰고 none으로 반환
        rows = []
        for row in csvreader:
            rows.append(row)

    # Global Variable : 함수 구축 시 주요하게 사용되는 변수를 확인하기 위한 장점때문에 사용
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 10, 1 # 독립변수의 크기와 종속변수의 크기
    
    # Buffer
    data = np.zeros([len(rows), input_cnt + output_cnt])

    # One-hot encoding
    for index, row in enumerate(rows):
        if row[0] == 'I' : data[index, 0] = 1
        elif row[0] == 'M' : data[index, 1] = 1
        elif row[0] == 'F' : data[index, 2] = 1
        data[index, 3:] = row[1:]

In [5]:
# Parameter Initialization
def init_model():
    global weight, bias, input_cnt, output_cnt
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt])
    bias = np.zeros([output_cnt])

In [6]:
# Train and Test / Result check
def train_and_test(epochs, mb_size, report, tr):
    steps = arrange_data(mb_size, tr)           # 반환하는 값은 미니배치가 몇 덩어리(스텝)으로 쪼개지는지?
    test_x, test_y = get_test_data()               # 테스트 데이터에 대한 독립변수와 종속변수를 얻어내는 함수

    # epochs는 외부 for문으로 돌림
    # 미니배치는 내부 for문으로 돌리기 때문에 시간복잡도 증가
    for epoch in range(epochs):
        losses, accuracies = [], [] # epochs당 손실함수와 정확도 for 평균
        for n in range(steps):
            train_x, train_y = get_train_data(mb_size, n) # 미니배치 사이즈와 스텝의 수만큼 개별값을 받아 학습데이터의 독립, 종속변수 반환
            loss, accuracy = run_train(train_x, train_y)
            losses.append(loss)
            accuracies.append(accuracy)
        
        if report > 0 and (epoch + 1) % report == 0:
            accuracy = run_test(test_x, test_y)
            print('Epoch {} : loss = {:5.3f}, accuracy = {:5.3f}, Test={:5.3f}'\
                .format(epoch + 1, np.mean(losses), np.mean(accuracies), accuracy))
    
    final_accuracy = run_test(test_x, test_y)
    print("\n 최종 테스트 결과 : final accuracy = {:5.3f}".format(final_accuracy))

In [7]:
# Datasets Shuffle -> mini batch size
def arrange_data(mb_size, tr):
    global data, shuffle_map, test_begin_index
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    # 미니 배치 스텝 수를 구하는 과정
    steps = int(data.shape[0] * tr) // mb_size

    # Search boundary line
    test_begin_index = steps * mb_size
    return steps

In [8]:
# Training Data
def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_index, output_cnt
    if nth== 0:
        np.random.shuffle(shuffle_map[:test_begin_index]) # 무작위로 특정 인덱스까지 무작위로 섞기
    train_data = data[shuffle_map[mb_size * nth : mb_size * (nth + 1)]]
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

In [9]:
# Test Data
def get_test_data():
    global data, shuffle_map, test_begin_index, output_cnt
    test_data = data[shuffle_map[test_begin_index:]]
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]

In [10]:
# Running Training
def run_train(x, y):
    output, aux_nn = forward_neuralnet(x)       # 신경망 연산, aux는 보조자료를 의미
    loss, aux_pp = forward_postproc(output, y)  # 신경망에 따른 손실함수 연산 과정('Mean Square Error'), aux 뒤 pp는 postprocessing
    
    # 학습 1단계(순전파 및 학습과정)
    accuracy = eval_accuracy(output, y)         

    # 학습 2단계
    G_loss = 1.0
    G_output = backprop_postproc(G_loss, aux_pp)
    backprop_neuralnet(G_output, aux_nn)

    return loss, accuracy

In [11]:
# Running Test
def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

In [12]:
# Evaluating Accuracy
def eval_accuracy(output, y):
    mdiff = np.mean(np.abs((output-y) / y))
    return 1 - mdiff

In [13]:
# Forward Propagation Neural Network : 신경망 순전파 연산
def forward_neuralnet(x):
    global weight, bias
    output = np.matmul(x, weight) + bias
    # 두 번째 반환 값인 x는 aux_nn으로 반환처리, 역전파 수행시 활용하기 위해 정의
    return output, x

In [14]:
# Forward Propagation Postprocessing : 신경망의 순전파에 따른 Mean Square Error를 도출하는 연산 
def forward_postproc(output, y):
    # 편차(diff) -> 제곱 -> 손실평균(loss)
    diff = output - y
    square = np.square(diff)

    loss = np.mean(square)
    # 편차를 반환하는 이유는 역전파 수행시 필요함(aux_pp)
    return loss, diff

In [15]:
# Backpropagation Postprocessing : 신경망의 역전파에 따른 선행 연산
def backprop_postproc(G_loss, diff):
    shape = diff.shape # M, N 정보 들어가 있음.
    g_loss_square = np.ones(shape) / np.prod(shape)
    g_square_diff = diff * 2
    g_diff_output = 1

    G_square = g_loss_square * G_loss
    G_diff = g_square_diff * G_square
    G_output = g_diff_output * G_diff

    return G_output

In [16]:
# Backpropagation Neural Network : 신경망 역전파 연산 -> 가중치, 편향 값 변화
def backprop_neuralnet(G_output, x):
    global weight, bias
    g_output_w = x.transpose() # 행렬 곱을 위한 transpose

    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis = 0)

    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

In [17]:
# main 함수 동작 구간
if __name__ == "__main__":
    regression_exec()

    print('\n가중치 :\n', weight)
    print('\n편향 :', bias)

Epoch 1 : train - loss = 37.336, accuracy = 0.523, Test=0.795
Epoch 2 : train - loss = 8.798, accuracy = 0.817, Test=0.818
Epoch 3 : train - loss = 7.675, accuracy = 0.814, Test=0.811
Epoch 4 : train - loss = 7.552, accuracy = 0.809, Test=0.810
Epoch 5 : train - loss = 7.473, accuracy = 0.809, Test=0.809
Epoch 6 : train - loss = 7.406, accuracy = 0.807, Test=0.811
Epoch 7 : train - loss = 7.351, accuracy = 0.809, Test=0.810
Epoch 8 : train - loss = 7.299, accuracy = 0.809, Test=0.810
Epoch 9 : train - loss = 7.252, accuracy = 0.808, Test=0.811
Epoch 10 : train - loss = 7.213, accuracy = 0.809, Test=0.811

 최종 테스트 결과 : final accuracy = 0.811

가중치 :
 [[1.03216701]
 [1.49331302]
 [1.58950247]
 [2.0296901 ]
 [1.59757436]
 [0.58972833]
 [2.44555537]
 [0.61032502]
 [0.49847128]
 [1.02251065]]

편향 : [4.1136748]
